In [2]:
import re, os
import json,sys
import csv
import collections
import itertools
import pandas as pd
pd.options.display.max_columns=50
from matplotlib import pyplot as plt

In [27]:
os.chdir(r"Z:\Shared Data\IRS-SOI\County-to-County-Migration\1516migrationdata")

df2=pd.read_csv("countyoutflow1516.csv")
# add foreign inflows
dfin=pd.read_csv("countyinflow1516.csv")
dfin=dfin[dfin.y1_statefips==98]
df2 = df2.append(dfin).reset_index(drop=True)

# State or Foreign (country) obs
df2=df2[(df2.y2_statefips.between(1,56) & df2.y2_countyfips>0) | (df2.y2_statefips ==98) ] 
df2=df2[(df2.y1_statefips.between(1,56) & df2.y1_countyfips>0) | (df2.y1_statefips ==98)]
# removes same county obs
df2=df2[(df2.y1_statefips*1000+df2.y1_countyfips != df2.y2_statefips*1000+df2.y2_countyfips)  ]
# missings were coded as -1. Remove them
df2=df2[df2.n1>0]
# Create state+county fips as ID.
df2["id2"]=df2.y2_statefips*1000+df2.y2_countyfips
df2["id1"]=df2.y1_statefips*1000+df2.y1_countyfips

# Add the inflows to each pair. Used to create net and check in and out at the same time.
df=df2.join(df2.set_index(['id2', 'id1']), on=['id1', 'id2'], how='left', rsuffix='in')
df.n1in=df.n1in.fillna(0)

# Find which states have little activity to group them into one other group.
route_counts_state = df.groupby(['y1_statefips']).sum().reset_index()
route_counts_state.loc[(route_counts_state.n1<150000) & (route_counts_state.n1in<150000) ,"AGG"]=1
route_counts_state.loc[(route_counts_state.y1_statefips==98) ,"AGG"]=0
#print(route_counts_state.AGG.value_counts())

df=df.join(route_counts_state[['y1_statefips','AGG']].set_index(['y1_statefips']), on=['y1_statefips'], how='left', rsuffix='st1')
df=df.join(route_counts_state[['y1_statefips','AGG']].set_index(['y1_statefips']), on=['y2_statefips'], how='left', rsuffix='st2')

# Add census regions
cenregions=pd.read_excel(r'Z:\Shared Data\IRS-SOI\County-to-County-Migration\SAS Programs\state-geocodes-v-regions2011.xls')
df=df.join(cenregions[['State\n(FIPS)','Region']].set_index(['State\n(FIPS)']), on=['y2_statefips'], how='left', rsuffix='y2')
df=df.join(cenregions[['State\n(FIPS)','Region']].set_index(['State\n(FIPS)']), on=['y1_statefips'], how='left', rsuffix='y1')


#print("DF HEAD:")
#print(df.head(10))

# generate R99 and R999SS ids for the other group. When state is 99, the counties are all in one state group. (Undoing the above.)
mask = (df.AGG==1) 
df.loc[mask,"id1"]=df.Regiony1*100000 + 99900 + df.y1_statefips
df.loc[mask,"y1_statefips"]=df.Regiony1*100 + 99
mask = (df.AGGst2==1) 
df.loc[mask,"id2"]=df.Region*100000 + 99900 + df.y2_statefips
df.loc[mask,"y2_statefips"]=df.Region*100 + 99



#print(pd.crosstab(df.y1_statefips,df.Regiony1))

# replace ids of small counties with SS999 for other.
route_counts_county1 = df.groupby(['id1']).sum().reset_index()
route_counts_county2 = df.groupby(['id2']).sum().reset_index()
route_counts_county2.rename(index=str, columns={"id2": "id1", "n1": "n1in", "n1in": "n1"})
route_counts_county1.append(route_counts_county2)
route_counts_county = route_counts_county1.groupby(['id1']).mean().reset_index()

## add in state aggregates to define small as less than 50K or 1/10 of migration
route_counts_county.y1_statefips=round(route_counts_county.id1/1000)
route_counts_county=route_counts_county.join(
    route_counts_state[['y1_statefips','n1','n1in']].set_index(['y1_statefips']),
    on=['y1_statefips'], how='left', rsuffix='st1')

# counties with migration less than min(50K , statemigration / 10) will be aggregated.
route_counts_county['test']=50000
route_counts_county.loc[route_counts_county.n1st1/10<50000,'test']=route_counts_county.n1st1/10
route_counts_county['test1']=50000
route_counts_county.loc[route_counts_county.n1inst1/10<50000,'test1']=route_counts_county.n1st1/10
mask=(route_counts_county.n1< route_counts_county.test) & (route_counts_county.n1in<route_counts_county.test1)
route_counts_county.loc[mask ,"AGGC"]=1
#print(route_counts_county.AGGC.value_counts())

df=df.join(route_counts_county[['id1','AGGC']].set_index(['id1']), on=['id1'], how='left', rsuffix='st1')
df=df.join(route_counts_county[['id1','AGGC']].set_index(['id1']), on=['id2'], how='left', rsuffix='st2')

mask = (df.AGGC==1) & (df.y1_statefips<99)
df.loc[mask,"id1"]=df.y1_statefips*1000+999
mask = (df.AGGCst2==1) & (df.y2_statefips<99) 
df.loc[mask,"id2"]=df.y2_statefips*1000+999

#print(route_counts_county[(route_counts_county.id1==13303)])


# Add State to County and State to State Migration
route_counts_state = df.groupby(['y1_statefips', 'y2_statefips']).sum().reset_index()
route_counts_state.id1=route_counts_state.y1_statefips*1000
route_counts_state.id2=route_counts_state.y2_statefips*1000
route_counts_state1 = df.groupby(['y1_statefips','id2']).sum().reset_index()
route_counts_state1.id1=route_counts_state1.y1_statefips*1000
route_counts_state2 = df.groupby(['id1', 'y2_statefips']).sum().reset_index()
route_counts_state2.id2=route_counts_state2.y2_statefips*1000

#print(route_counts_state[route_counts_state.y1_statefips==99].head())

route_counts = df.groupby(['id1', 'id2']).sum().reset_index()
route_counts = route_counts[route_counts.n1>0]

route_counts=route_counts.append(route_counts_state)
route_counts=route_counts.append(route_counts_state1)
route_counts=route_counts.append(route_counts_state2)

# replace 98000 with 998000 for better sort
route_counts.loc[(route_counts.id1==98000) ,"id1"]=998000
route_counts.loc[(route_counts.id2==98000) ,"id2"]=998000
# add an artificial subgroup for foreign
temp=route_counts[(route_counts.id1==998000)|(route_counts.id2==998000)]
temp.loc[(temp.id1==998000) ,"id1"]=998999
temp.loc[(temp.id2==998000) ,"id2"]=998999
route_counts=route_counts.append(temp)

# create net migration file.
route_counts['n1_net']=route_counts.n1-route_counts.n1in
route_counts_net=route_counts[route_counts.n1_net>0]

route_counts[route_counts.id2==998999].head()
#print(len(route_counts.index),len(df.index))
#route_counts.tail(5)

#route_counts_state.y1_statefips.value_counts()

# To figure out which states to aggregate, let's see the histogram of n1 by state.
#plt.close()
#fig1=plt.figure(figsize=(17,10))
#route_counts_state.hist(column='n1')
#plt.xlabel("Trip_distance",fontsize=15)
#plt.ylabel("Frequency",fontsize=15)
#plt.xlim([0.0,40000.0])
#plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,AGG,AGGC,AGGCst2,AGGst2,Region,Regiony1,agi,agiin,id1,id2,n1,n1in,n2,n2in,y1_countyfips,y1_countyfipsin,y1_statefips,y1_statefipsin,y2_countyfips,y2_countyfipsin,y2_statefips,y2_statefipsin,n1_net
47,NaN,NaN,NaN,0.0,NaN,4.0,212470,225288.0,6037.0,998999.0,2428,1813.0,4236,3454.0,37,0.0,6.0,98.0,0,37.0,98.0,6.0,615.0
131,NaN,NaN,NaN,0.0,NaN,4.0,80177,58068.0,6059.0,998999.0,810,577.0,1549,1320.0,59,0.0,6.0,98.0,0,59.0,98.0,6.0,233.0
208,NaN,NaN,NaN,0.0,NaN,4.0,155626,153217.0,6073.0,998999.0,2663,3075.0,5783,6113.0,73,0.0,6.0,98.0,0,73.0,98.0,6.0,-412.0
292,NaN,33.0,NaN,0.0,NaN,132.0,732008,561828.0,6999.0,998999.0,7028,4930.0,13879,10406.0,2153,0.0,198.0,2940.0,0,2044.0,3234.0,180.0,2098.0
344,NaN,NaN,NaN,0.0,NaN,4.0,2597,2039.0,8001.0,998999.0,67,48.0,132,90.0,1,0.0,8.0,98.0,0,1.0,98.0,8.0,19.0


In [29]:
measures={0: route_counts, 1: route_counts_net}
for i in range(2):
    measures.get(i)

# Turn the bilateral observations into a matrix
# Save the location of variables of interest
    locid2=measures.get(i).columns.get_loc("id2")
    locid1=measures.get(i).columns.get_loc("id1")
    if i==0:
        locn1=measures.get(i).columns.get_loc("n1")
    else:
        locn1=measures.get(i).columns.get_loc("n1_net")

    grid = collections.Counter()

    for index,line in measures.get(i).iterrows():
        # clean empty names
        line = [name for name in line]
        #print(line)
        # do pairwise stuff
        if str(line[locid1])==str(line[locid2]):
            grid[(int(line[locid1]),int(line[locid2]))] = '0'   
        else:
            grid[(int(line[locid1]),int(line[locid2]))] = line[locn1]
        #grid[pair[::-1]] += 1

    actors = sorted(set(pair[0] for pair in grid))

    fips=[k for k in range(56)]
    fips.extend([199,299,399,499,998])
    regions=[]
    for l in fips:
        try:
            regions.append(actors.index(l*1000))
        except:
            pass
            #print(str(l),"state not found")
    print("Indices of state aggregates are", regions)
    print("regions are:" , [actors[k] for k in regions])



    locals()["mat1516"+str(i)]=[]
    with open("connection_grid"+str(i)+".csv", "w", newline="") as fp:
        writer = csv.writer(fp)
        writer.writerow([''] + actors)
        for actor in actors:
            line = [actor,] + [int(grid[actor, other]) for other in actors]
            locals()["mat1516"+str(i)].append(line[1:])
            writer.writerow(line)
    #print('\n'.join([''.join(['{:4},'.format(item) for item in row[0:10]]) 
    #      for row in mat1516[0:10]]))
    #print(mat1516[0])


Indices of state aggregates are [0, 5, 12, 15, 21, 25, 29, 34, 39, 43, 47, 51, 56, 61, 68, 78, 92, 102]
regions are: [6000, 8000, 12000, 13000, 17000, 26000, 34000, 36000, 37000, 39000, 42000, 48000, 53000, 199000, 299000, 399000, 499000, 998000]
Indices of state aggregates are [0, 5, 12, 15, 21, 25, 29, 34, 39, 43, 47, 51, 56, 61, 68, 78, 92, 102]
regions are: [6000, 8000, 12000, 13000, 17000, 26000, 34000, 36000, 37000, 39000, 42000, 48000, 53000, 199000, 299000, 399000, 499000, 998000]


In [37]:
# Create the Labels
os.chdir(r"Z:\Shared Data\IRS-SOI\County-to-County-Migration\1516migrationdata")
df2=pd.read_csv("countyoutflow1516.csv")

## remove extra words
df2.y2_countyname=df2.y2_countyname.str.replace(" County","")
df2.y2_countyname=df2.y2_countyname.str.replace(" Non-migrants","")
# get id to label data set
dd=df2.groupby(['y2_statefips','y2_countyfips','y2_state','y2_countyname']).size().reset_index().rename(columns={0:'count'})
dd['id'] = dd.y2_statefips*1000 + dd.y2_countyfips
dd['label1'] = dd.y2_countyname + ", " + dd.y2_state
# generate labels for states and region+state observations
ss=df2.groupby(['y2_statefips','y2_state']).size().reset_index().rename(columns={0:'count'})
ss=ss[ss.y2_statefips<57]
ss['label1'] = ss.y2_state
for i in range(1,5):
    locals()["ss"+str(i)]=ss.copy(deep=True)
    locals()["ss"+str(i)]['id']= (i*100000)+ 99900 + locals()["ss"+str(i)]['y2_statefips']
ss['id'] = ss.y2_statefips*1000 
# labels for other counties
ss_other=ss.copy(deep=True)
ss_other['id'] = ss_other.id+999
ss_other['label1'] = "Other Counties, "+ss_other.label1
# for regions
cenreg=cenregions.drop_duplicates(subset=['Region'])
cenreg['id']= (cenreg.Region*100+99)*1000
cenreg['label1']="States in "+ cenreg.Name.str.replace("Northeast","NE")
# for foreign
matrixF={}
matrixF['id']=[998000,998999]
matrixF['label1']=["Foreign","Foreign"]

F=pd.DataFrame(matrixF)

print(F.head())
dd=dd.append(cenreg)
dd=dd.append(ss).append(ss1).append(ss2).append(ss3).append(ss4)
dd=dd.append(ss_other)
dd=dd.append(F)

dd = dd[['id','label1']].drop_duplicates(subset='id')

#testing
#print(df2[(df2.y2_statefips==8)&(df2.y2_countyfips==25)])

actors1=pd.DataFrame(data=actors,    # values
                   columns=['id'], dtype='int32')  # 1st row as the column names
#print(actors1.head())
actors2=actors1.join(dd.set_index(['id']), on=['id'], how='left')
print(actors2.head(20))

names= actors2.label1.tolist()
migration={}
migration={'names':names,'regions': regions,'matrix':{'2015-16':mat15160, '2015-16 (Net)':mat15161}}

#test
migration.keys()
[migration['names'][i] for i in migration['regions']]



       id   label1
0  998000  Foreign
1  998999  Foreign
       id              label1
0    6000                  CA
1    6037     Los Angeles, CA
2    6059          Orange, CA
3    6073       San Diego, CA
4    6999  Other Counties, CA
5    8000                  CO
6    8001           Adams, CO
7    8005        Arapahoe, CO
8    8031          Denver, CO
9    8041         El Paso, CO
10   8059       Jefferson, CO
11   8999  Other Counties, CO
12  12000                  FL
13  12095          Orange, FL
14  12999  Other Counties, FL
15  13000                  GA
16  13067            Cobb, GA
17  13089          DeKalb, GA
18  13121          Fulton, GA
19  13135        Gwinnett, GA


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['CA',
 'CO',
 'FL',
 'GA',
 'IL',
 'MI',
 'NJ',
 'NY',
 'NC',
 'OH',
 'PA',
 'TX',
 'WA',
 'States in NE Region',
 'States in Midwest Region',
 'States in South Region',
 'States in West Region',
 'Foreign']

In [38]:
os.chdir(r"Z:\Evaluations\Migration\State-County-migration-plot\docs")
with open('migration.json', 'w') as outfile:
    json.dump(migration, outfile)